1. Importa bibliotecas `pandas` e `numpy`.  
2. Define semente aleatória para reprodutibilidade.  
3. Cria base fictícia com 1000 transações.  
4. Gera valores aleatórios para transações, taxas e isenções.  
5. Corrige taxas para zero quando há isenção aplicada.  
6. Introduz 5 erros simulando cobranças indevidas em isentos.  
7. Insere 5 anomalias com taxas muito altas em transações não isentas.  
8. Cria DataFrame com os dados simulados.  
9. Salva os dados em um arquivo CSV.  
10. Exibe mensagem de sucesso após gerar o arquivo.

In [ ]:
import pandas as pd
import numpy as np

# 1. Criar base de dados fictícia
np.random.seed(42)
data = {
    "ID_Transação": range(1, 1001),
    "Valor_Transação": np.random.uniform(50, 500, 1000).round(2),
    "Taxa_Cobrada": np.random.uniform(1, 15, 1000).round(2),
    "Isenção_Aplicada": np.random.choice(["Sim", "Não"], 1000, p=[0.2, 0.8])
}
df = pd.DataFrame(data)

# 2. Simular erros e anomalias
# 2.1. Corrigir isenções (taxa = 0) e adicionar erros de cobrança
df.loc[df["Isenção_Aplicada"] == "Sim", "Taxa_Cobrada"] = 0
erros_isenção = np.random.choice(df[df["Isenção_Aplicada"] == "Sim"].index, 5, replace=False)
df.loc[erros_isenção, "Taxa_Cobrada"] = np.random.uniform(1, 10, 5).round(2)

# 2.2. GERAR ANOMALIAS (outliers) em transações NÃO isentas
anomalias = np.random.choice(df[df["Isenção_Aplicada"] == "Não"].index, 5, replace=False)
df.loc[anomalias, "Taxa_Cobrada"] = np.random.choice([45, 46, 47, 48, 49], 5)  # Valores extremos

# 3. Salvar em CSV (arquivo será criado no mesmo diretório do script)
df.to_csv("transacoes_paysmart.csv", index=False, encoding='utf-8-sig')

print("✅ Arquivo 'transacoes_paysmart.csv' gerado com sucesso!")

xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx

1. Importa bibliotecas para manipulação de dados, detecção de anomalias (`IsolationForest`), padronização (`StandardScaler`), salvamento de modelos (`joblib`) e controle de tempo.  
2. Define constantes com os nomes do arquivo de dados (`transacoes_paysmart.csv`) e do modelo salvo (`modelo_cobrancas.pkl`).  
3. Prepara o ambiente para carregar dados, treinar ou usar modelo de machine learning em cobrança.

In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
import joblib
import os
from datetime import datetime
import time

# Nome do arquivo (coloque na mesma pasta do script)
ARQUIVO_DADOS = "transacoes_paysmart.csv"  
MODELO_PATH = "modelo_cobrancas.pkl"

1. Lê os dados do arquivo CSV definido (`ARQUIVO_DADOS`).  
2. Cria a variável `Taxa_Relativa` e seleciona colunas relevantes para o modelo.  
3. Normaliza os dados com `StandardScaler` e treina um `IsolationForest` para detectar anomalias.  
4. Salva o modelo e o scaler em um arquivo (`MODELO_PATH`) com `joblib`.

In [3]:
def treinar_modelo():
    """Treina o modelo com os dados disponíveis"""
    df = pd.read_csv(ARQUIVO_DADOS)
    
    # Preparação dos dados
    df['Taxa_Relativa'] = df['Taxa_Cobrada'] / df['Valor_Transação']
    features = ['Valor_Transação', 'Taxa_Cobrada', 'Taxa_Relativa']
    X = df[features]
    
    # Normalização e modelo
    scaler = StandardScaler().fit(X)
    model = IsolationForest(contamination=0.005, random_state=42).fit(scaler.transform(X))
    
    # Salva modelo e scaler juntos
    joblib.dump({'model': model, 'scaler': scaler}, MODELO_PATH)
    return model, scaler

1. Verifica se o arquivo do modelo (`MODELO_PATH`) existe.  
2. Tenta carregar o modelo e o scaler com `joblib`.  
3. Se carregar com sucesso, retorna o modelo e o scaler salvos.  
4. Em caso de erro ou ausência do arquivo, treina e retorna um novo modelo.

In [4]:
def carregar_modelo():
    """Carrega o modelo se existir, ou treina um novo"""
    if os.path.exists(MODELO_PATH):
        try:
            dados = joblib.load(MODELO_PATH)
            print("✅ Modelo carregado")
            return dados['model'], dados['scaler']
        except:
            print("⚠️ Erro ao carregar, treinando novo...")
    return treinar_modelo()

1. Carrega ou treina o modelo e o scaler com `carregar_modelo()`.  
2. Lê os dados do arquivo CSV com transações.  
3. Calcula a coluna `Taxa_Relativa` para análise.  
4. Padroniza os dados com o scaler carregado.  
5. Usa o modelo `IsolationForest` para identificar anomalias.  
6. Detecta erros de isenção com base nas regras de cobrança.  
7. Filtra as transações com anomalias ou erros.  
8. Se houver problemas, salva um relatório em CSV com data e hora.  
9. Imprime mensagem com número de alertas ou confirmação de normalidade.  
10. Executa a função `monitorar()` em loop, com pausa de 30 segundos entre execuções.

In [ ]:
def monitorar():
    """Executa a detecção de anomalias"""
    model, scaler = carregar_modelo()
    df = pd.read_csv(ARQUIVO_DADOS)
    
    # Prepara e classifica os dados
    df['Taxa_Relativa'] = df['Taxa_Cobrada'] / df['Valor_Transação']
    X = scaler.transform(df[['Valor_Transação', 'Taxa_Cobrada', 'Taxa_Relativa']])
    df['Anomalia'] = np.where(model.predict(X) == -1, 1, 0)
    df['Erro_Isencao'] = (df['Isenção_Aplicada'] == "Sim") & (df['Taxa_Cobrada'] > 0)
    
    # Filtra problemas
    problemas = df[df['Anomalia'] | df['Erro_Isencao']]
    if not problemas.empty:
        nome_relatorio = f"alertas_{datetime.now().strftime('%Y%m%d_%H%M')}.csv"
        problemas.to_csv(nome_relatorio, index=False)
        print(f"⚠️ {len(problemas)} alertas em {nome_relatorio}")
    else:
        print("✅ Nenhum problema detectado")

# Executa a cada hora
while True:
    monitorar()
    time.sleep(30)